In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # Upgrade pip
!pip install --upgrade pip --quiet

# Install/upgrade required libraries
!pip install torch --quiet
!pip install torchvision --quiet
!pip install torchaudio --quiet
!pip install transformers==4.33.1 --quiet
!pip install datasets --quiet
!pip install scikit-learn --quiet
!pip install codecarbon --quiet
!pip install numpy==1.26.4 --quiet
!pip install pandas --quiet
!pip install tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
error: failed-wheel-build-for-install

× Failed to build installable wheels for some pyproject.toml based projects
╰─> tokenizers
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.38.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which 

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification,  DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers.models.bert.modeling_bert import BertAttention, BertIntermediate, BertOutput
from transformers import DistilBertForSequenceClassification, DistilBertModel
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
import os
import time
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers.data.data_collator import default_data_collator
import logging
import warnings
import random
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import json
from codecarbon import OfflineEmissionsTracker
import math

# --- Configuration ---
warnings.filterwarnings("ignore")
logging.getLogger("codecarbon").setLevel(logging.WARNING)
DEVICE_CONFIG = {
    'optimize_for_gpu': True,
    'mixed_precision': True
}
DEVICE = "cuda" if torch.cuda.is_available() and DEVICE_CONFIG['optimize_for_gpu'] else "cpu"
print(f"Using device: {DEVICE}")
DATASETS = {
    'glue_sst2': {'name': 'glue', 'config': 'sst2', 'split': 'validation'},
    'glue_mrpc': {'name': 'glue', 'config': 'mrpc', 'split': 'validation'},
    'glue_rte': {'name': 'glue', 'config': 'rte', 'split': 'validation'}
}
MAX_SAMPLES = 5000
WATER_USAGE_FACTORS = {"average_l_per_kwh": 1.8}
CARBON_INTENSITY = 250  # gCO2e/kWh
GLUE_TASKS_TO_RUN = ['sst2', 'mrpc', 'rte']
BATCH_SIZE = 16
SEQ_LEN = 128
NUM_LAYERS = 1
TIME_STEPS = 8  # Reduced from 16
PATIENCE = 2  # Early stopping patience


default_data_collator = lambda x: {k: torch.stack([torch.tensor(f[k]) for f in x]) for k in x[0]}

# --- Utility Functions ---
def get_device():
    device = torch.device(DEVICE)
    if DEVICE == "cuda":
        try:
            print(f"✓ Using GPU: {torch.cuda.get_device_name(0)} (CUDA)")
        except Exception:
            print("✓ Using GPU (name unknown)")
        if DEVICE_CONFIG['mixed_precision']:
            print("✓ Mixed precision (FP16) enabled.")
    else:
        print(f"✓ Using CPU")
    return device

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def tensor_to_spikes(tensor, time_steps=TIME_STEPS):
    """Converts a normalized tensor to a spike train with sparsity."""
    device = tensor.device
    dtype = tensor.dtype
    random_tensor = torch.rand(*tensor.shape, time_steps, device=device, dtype=dtype)
    spikes = (random_tensor < tensor.unsqueeze(-1)).to(dtype)
    # Introduce sparsity: only keep top 80% of spikes
    threshold = torch.quantile(spikes.float(), 0.2, dim=-1, keepdim=True)
    spikes = spikes * (spikes >= threshold).to(dtype)
    return spikes

def spikes_to_tensor(spikes):
    """Converts a spike train back to a tensor by averaging over the time dimension."""
    return torch.mean(spikes, dim=-1)

# --- Optimized Spiking Self-Attention ---
class SpikingSelfAttention(nn.Module):
    """Optimized SNN-based self-attention with sparsity and reduced time steps."""
    def __init__(self, hidden_dim, num_heads, device=None, sparsity=0.5):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.device = device or DEVICE
        self.sparsity = sparsity
        self.query_layer = nn.Linear(hidden_dim, hidden_dim).to(self.device)
        self.key_layer = nn.Linear(hidden_dim, hidden_dim).to(self.device)
        self.value_layer = nn.Linear(hidden_dim, hidden_dim).to(self.device)
        self.output_layer = nn.Linear(hidden_dim, hidden_dim).to(self.device)
        print("✓ Initialized optimized SNN attention module with sparsity")

    def forward(self, spike_input):
        input_tensor = spikes_to_tensor(spike_input)
        target_dtype = next(self.query_layer.parameters()).dtype
        if input_tensor.dtype != target_dtype:
            input_tensor = input_tensor.to(target_dtype)
        query = self.query_layer(input_tensor)
        key = self.key_layer(input_tensor)
        value = self.value_layer(input_tensor)
        attention_weights = torch.softmax(torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.hidden_dim), dim=-1)
        # Apply sparsity to attention weights
        threshold = torch.quantile(attention_weights, self.sparsity, dim=-1, keepdim=True)
        attention_weights = attention_weights * (attention_weights >= threshold).to(attention_weights.dtype)
        attention_output = torch.matmul(attention_weights, value)
        output = self.output_layer(attention_output)
        return tensor_to_spikes(output)

# --- Optimized Hybrid Layers ---
class HybridBertLayer(nn.Module):
    """Optimized hybrid layer for BERT with dynamic SNN attention usage."""
    def __init__(self, original_layer, device=None, sparsity=0.5):
        super().__init__()
        self.device = device or DEVICE
        self.original_layer = original_layer
        attention_module = original_layer.attention
        config = attention_module.self.query.in_features
        num_heads = attention_module.self.num_attention_heads
        self.snn_attention = SpikingSelfAttention(hidden_dim=config, num_heads=num_heads, device=self.device, sparsity=sparsity)
        self.use_snn = True

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_values=None,
        output_attentions=False,
        output_hidden_states=False,
        **kwargs
    ):
        norm = torch.norm(hidden_states, dim=-1).mean()
        self.use_snn = norm > 0.5
        if self.use_snn:
            attention_output = self.original_layer.attention.output.LayerNorm(hidden_states)
            input_spikes = tensor_to_spikes(attention_output)
            snn_attention_output_tensor = spikes_to_tensor(self.snn_attention(input_spikes))
            attention_output = snn_attention_output_tensor + hidden_states
        else:
            attention_output = self.original_layer.attention(hidden_states, attention_mask, head_mask)[0]
        intermediate_output = self.original_layer.intermediate(attention_output)
        layer_output = self.original_layer.output(intermediate_output, attention_output)
        return (layer_output,)

class HybridDistilBertLayer(nn.Module):
    """Optimized hybrid layer for DistilBERT with dynamic SNN attention usage."""
    def __init__(self, original_layer, device=None, sparsity=0.5):
        super().__init__()
        self.device = device or DEVICE
        self.original_layer = original_layer
        config = original_layer.attention.q_lin.in_features
        num_heads = original_layer.attention.n_heads
        self.snn_attention = SpikingSelfAttention(hidden_dim=config, num_heads=num_heads, device=self.device, sparsity=sparsity)
        self.ffn = original_layer.ffn
        self.sa_layer_norm = original_layer.sa_layer_norm
        self.output_layer_norm = original_layer.output_layer_norm
        self.use_snn = True

    def forward(self, x, attn_mask=None, head_mask=None, output_attentions=False):
        norm = torch.norm(x, dim=-1).mean()
        self.use_snn = norm > 0.5
        if self.use_snn:
            normed_states = self.sa_layer_norm(x)
            input_spikes = tensor_to_spikes(normed_states)
            attention_spikes = self.snn_attention(input_spikes)
            attention_output_tensor = spikes_to_tensor(attention_spikes)
            attention_output = attention_output_tensor + x
        else:
            attention_output = self.original_layer(x, attn_mask=attn_mask, head_mask=head_mask, output_attentions=output_attentions)[0]
        ffn_input = self.output_layer_norm(attention_output)
        layer_output = self.ffn(ffn_input)
        return (layer_output,)

# --- Build Hybrid Model ---
def build_hybrid_neuromorphic_model(device, model_type="bert", task_type="classification", num_layers=1, debug=False):
    """Builds an optimized hybrid neuromorphic BERT or DistilBERT model."""
    print(f"\n🏗️ Building Optimized Hybrid Neuromorphic {model_type.capitalize()} Model for {task_type}...")
    if model_type == "bert":
        if task_type == "classification":
            base_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
        else:
            raise ValueError(f"Unsupported task_type {task_type} for BERT")
        original_layers = base_model.bert.encoder.layer
        layer_class = HybridBertLayer
    elif model_type == "distilbert":
        if task_type == "classification":
            base_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
        else:
            raise ValueError(f"Unsupported task_type {task_type} for DistilBERT")
        original_layers = base_model.distilbert.transformer.layer
        layer_class = HybridDistilBertLayer
    else:
        raise ValueError(f"Unsupported model_type {model_type}")

    class HybridModel(nn.Module):
        def __init__(self):
            super().__init__()
            self.transformer = base_model.to(device)
            self.model_type = model_type
            self.task_type = task_type
            self.debug = debug
            for i in range(min(num_layers, len(original_layers))):
                original_layers[-(i + 1)] = layer_class(original_layers[-(i + 1)], device=device, sparsity=0.5)
            self.config = base_model.config
            self.transformer.gradient_checkpointing_enable()

        def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, **kwargs):
            inputs = {}
            if input_ids is not None:
                inputs["input_ids"] = input_ids
            if attention_mask is not None:
                inputs["attention_mask"] = attention_mask
            if token_type_ids is not None and self.model_type == "bert":
                inputs["token_type_ids"] = token_type_ids
            outputs = self.transformer(**inputs, **kwargs)
            if self.debug:
                print(f"[Hybrid Layers] Logits norm: {outputs.logits.norm().item():.4f}")
            return outputs

    model = HybridModel()
    print(f"✓ Successfully created optimized hybrid neuromorphic {model_type.capitalize()} model for {task_type}.")
    return model

# --- Data Loading ---
def get_dataloaders(model_type, seq_len, batch_size):
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    def preprocess_glue_sst2(examples):
        enc = tokenizer(examples["sentence"], padding="max_length", max_length=seq_len, truncation=True)
        return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "token_type_ids": enc["token_type_ids"], "labels": examples["label"]}

    def preprocess_glue_mrpc(examples):
        enc = tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", max_length=seq_len, truncation=True)
        return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "token_type_ids": enc["token_type_ids"], "labels": examples["label"]}

    def preprocess_glue_rte(examples):
        enc = tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", max_length=seq_len, truncation=True)
        return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "token_type_ids": enc["token_type_ids"], "labels": examples["label"]}

    try:
        glue_sst2_train_data = load_dataset("glue", "sst2", split=f"train[:{MAX_SAMPLES}]")
        glue_sst2_validation_data = load_dataset("glue", "sst2", split=f"validation[:{MAX_SAMPLES}]")
        glue_mrpc_train_data = load_dataset("glue", "mrpc", split=f"train[:{5000}]")
        glue_mrpc_validation_data = load_dataset("glue", "mrpc", split=f"validation[:{5000}]")
        glue_rte_train_data = load_dataset("glue", "rte", split=f"train[:{2490}]")
        glue_rte_validation_data = load_dataset("glue", "rte", split=f"validation[:{277}]")

        glue_sst2_train_data = glue_sst2_train_data.map(preprocess_glue_sst2, batched=True, remove_columns=['sentence', 'idx', 'label'])
        glue_sst2_validation_data = glue_sst2_validation_data.map(preprocess_glue_sst2, batched=True, remove_columns=['sentence', 'idx', 'label'])
        glue_mrpc_train_data = glue_mrpc_train_data.map(preprocess_glue_mrpc, batched=True, remove_columns=['sentence1', 'sentence2', 'idx', 'label'])
        glue_mrpc_validation_data = glue_mrpc_validation_data.map(preprocess_glue_mrpc, batched=True, remove_columns=['sentence1', 'sentence2', 'idx', 'label'])
        glue_rte_train_data = glue_rte_train_data.map(preprocess_glue_rte, batched=True, remove_columns=['sentence1', 'sentence2', 'idx', 'label'])
        glue_rte_validation_data = glue_rte_validation_data.map(preprocess_glue_rte, batched=True, remove_columns=['sentence1', 'sentence2', 'idx', 'label'])

        glue_sst2_train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"])
        glue_sst2_validation_data.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"])
        glue_mrpc_train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"])
        glue_mrpc_validation_data.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"])
        glue_rte_train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"])
        glue_rte_validation_data.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"])

        dataloaders = {
            'glue_sst2_train': DataLoader(glue_sst2_train_data, batch_size=batch_size, shuffle=True, collate_fn=default_data_collator),
            'glue_sst2_validation': DataLoader(glue_sst2_validation_data, batch_size=batch_size, shuffle=False, collate_fn=default_data_collator),
            'glue_mrpc_train': DataLoader(glue_mrpc_train_data, batch_size=batch_size, shuffle=True, collate_fn=default_data_collator),
            'glue_mrpc_validation': DataLoader(glue_mrpc_validation_data, batch_size=batch_size, shuffle=False, collate_fn=default_data_collator),
            'glue_rte_train': DataLoader(glue_rte_train_data, batch_size=batch_size, shuffle=True, collate_fn=default_data_collator),
            'glue_rte_validation': DataLoader(glue_rte_validation_data, batch_size=batch_size, shuffle=False, collate_fn=default_data_collator),
        }
        mask_token_id = tokenizer.convert_tokens_to_ids("[MASK]")
        return dataloaders, tokenizer, mask_token_id, batch_size, seq_len
    except Exception as e:
        print(f"⚠️ get_dataloaders failed: {e}. Returning empty dataloaders.")
        empty_data = {
            "input_ids": torch.zeros((0, seq_len), dtype=torch.long),
            "attention_mask": torch.zeros((0, seq_len), dtype=torch.long),
            "token_type_ids": torch.zeros((0, seq_len), dtype=torch.long),
            "labels": torch.tensor([], dtype=torch.long)
        }
        empty_dataset = Dataset.from_dict(empty_data)
        empty_dataloader = DataLoader(empty_dataset, batch_size=batch_size, collate_fn=default_data_collator)
        empty_dataloaders = {
            'glue_sst2_train': empty_dataloader, 'glue_sst2_validation': empty_dataloader,
            'glue_mrpc_train': empty_dataloader, 'glue_mrpc_validation': empty_dataloader,
            'glue_rte_train': empty_dataloader, 'glue_rte_validation': empty_dataloader,
        }
        mask_token_id = tokenizer.convert_tokens_to_ids("[MASK]")
        return empty_dataloaders, tokenizer, mask_token_id, batch_size, seq_len

# --- Fine-Tuning with Early Stopping ---
def fine_tune(model, train_dataloader, val_dataloader, device, epochs=1, task_type="classification", patience=PATIENCE):
    """Fine-tunes the model with gradient checkpointing, cosine annealing LR, and early stopping."""
    model.train()
    optimizer = AdamW(model.parameters(), lr=5e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs * len(train_dataloader))
    loss_fct = torch.nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    for epoch in range(epochs):
        total_train_loss = 0
        for batch in tqdm(train_dataloader, desc=f"Fine-tuning epoch {epoch+1}"):
            inputs = {k: v.to(device) for k, v in batch.items() if k not in ["labels"]}
            optimizer.zero_grad()
            outputs = model(**inputs)
            labels = batch["labels"].to(device)
            logits = outputs.logits
            loss = loss_fct(logits, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_dataloader) if len(train_dataloader) > 0 else 0
        print(f"Epoch {epoch+1}, Average Training Loss: {avg_train_loss:.4f}")

        # Validation step
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc=f"Validating epoch {epoch+1}"):
                inputs = {k: v.to(device) for k, v in batch.items() if k not in ["labels"]}
                labels = batch["labels"].to(device)
                outputs = model(**inputs)
                logits = outputs.logits
                loss = loss_fct(logits, labels)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_dataloader) if len(val_dataloader) > 0 else 0
        print(f"Epoch {epoch+1}, Average Validation Loss: {avg_val_loss:.4f}")

        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            patience_counter = 0
            print(f"New best validation loss: {best_val_loss:.4f}")
        else:
            patience_counter += 1
            print(f"No improvement in validation loss. Patience counter: {patience_counter}/{patience}")
            if patience_counter >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs.")
                model.load_state_dict(best_model_state)
                break

        model.train()

    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    return best_val_loss

# --- Evaluation ---
def evaluate_classification(model, dataloader, device, is_bert=True):
    """Evaluates the model on classification tasks."""
    model.eval()
    correct, total = 0, 0
    predictions_list, labels_list = [], []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            inputs = {k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask', 'token_type_ids']}
            labels = batch["labels"].to(device)
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
            predictions_list.extend(predictions.cpu().numpy())
            labels_list.extend(labels.cpu().numpy())

    accuracy = correct / total if total > 0 else 0.0
    f1 = f1_score(labels_list, predictions_list, average='weighted')
    return {'accuracy': accuracy, 'f1': f1}



# --- Experiment Runner ---
def run_experiment(model, model_name, dataloaders, device, is_bert, batch_size, seq_len, task_type, run_sst2=True, run_mrpc=True, run_rte=True):
    """Runs an optimized experiment with performance metrics."""
    results = {
        'model_name': model_name,
        'batch_size': batch_size,
        'seq_length': seq_len,
        'task_type': task_type,
        'accuracy_metrics': {},
        'performance_metrics': {}
    }
    print(f"\n--- 🚀 Measuring Metrics for {model_name} ({task_type}) ---")
    start_time = time.time()
    num_queries = 0
    emissions_kwh = 0.0

    tracker = OfflineEmissionsTracker(
        project_name=f"Experiment_{model_name.replace(' ', '_')}",
        measure_power_secs=1,
        output_dir=".",
        log_level='warning',
        country_iso_code="USA",
        region=None
    )
    tracker.start()

    try:
        if run_sst2 and task_type == "classification":
            print("\n--- Fine-tuning on GLUE SST-2 ---")
            best_val_loss = fine_tune(model, dataloaders['glue_sst2_train'], dataloaders['glue_sst2_validation'], device, epochs=8, task_type=task_type)
            print(f"Best Validation Loss for SST-2: {best_val_loss:.4f}")
            print(" Evaluating GLUE SST-2...")
            metrics = evaluate_classification(model, dataloaders['glue_sst2_validation'], device, is_bert)
            results['accuracy_metrics']['sst2_accuracy'] = metrics['accuracy']
            results['accuracy_metrics']['sst2_f1'] = metrics['f1']
            num_queries += len(dataloaders['glue_sst2_validation'].dataset)
            print(f" SST-2 Accuracy: {metrics['accuracy']:.4f}, F1: {metrics['f1']:.4f}")

        if run_mrpc and task_type == "classification":
            print("\n--- Fine-tuning on GLUE MRPC ---")
            best_val_loss = fine_tune(model, dataloaders['glue_mrpc_train'], dataloaders['glue_mrpc_validation'], device, epochs=5, task_type=task_type)
            print(f"Best Validation Loss for MRPC: {best_val_loss:.4f}")
            print(" Evaluating GLUE MRPC...")
            metrics = evaluate_classification(model, dataloaders['glue_mrpc_validation'], device, is_bert)
            results['accuracy_metrics']['mrpc_accuracy'] = metrics['accuracy']
            results['accuracy_metrics']['mrpc_f1'] = metrics['f1']
            num_queries += len(dataloaders['glue_mrpc_validation'].dataset)
            print(f" MRPC Accuracy: {metrics['accuracy']:.4f}, F1: {metrics['f1']:.4f}")

        if run_rte and task_type == "classification":
            print("\n--- Fine-tuning on GLUE RTE ---")
            best_val_loss = fine_tune(model, dataloaders['glue_rte_train'], dataloaders['glue_rte_validation'], device, epochs=3, task_type=task_type)
            print(f"Best Validation Loss for RTE: {best_val_loss:.4f}")
            print(" Evaluating GLUE RTE...")
            metrics = evaluate_classification(model, dataloaders['glue_rte_validation'], device, is_bert)
            results['accuracy_metrics']['rte_accuracy'] = metrics['accuracy']
            results['accuracy_metrics']['rte_f1'] = metrics['f1']
            num_queries += len(dataloaders['glue_rte_validation'].dataset)
            print(f" RTE Accuracy: {metrics['accuracy']:.4f}, F1: {metrics['f1']:.4f}")

        emissions_kwh = tracker.stop() or 0.0
    except Exception as e:
        print(f"🚨 Experiment failed for {model_name}: {e}")
        if tracker._running:
            emissions_kwh = tracker.stop() or 0.0
        else:
            emissions_kwh = 0.0

    total_duration_s = time.time() - start_time
    total_tokens_processed = num_queries * seq_len
    total_carbon_g = emissions_kwh * CARBON_INTENSITY
    results['performance_metrics'] = {
        'latency_ms_query': (total_duration_s / num_queries) * 1000 if num_queries > 0 else 0,
        'throughput_tokens_sec': total_tokens_processed / total_duration_s if total_duration_s > 0 else 0,
        'energy_wh_token': (emissions_kwh * 1000) / total_tokens_processed if total_tokens_processed > 0 else 0,
        'sci_gco2e_query': total_carbon_g / num_queries if num_queries > 0 else 0,
        'wue_avg_liters_query': (emissions_kwh * WATER_USAGE_FACTORS['average_l_per_kwh']) / num_queries if num_queries > 0 else 0,
        'total_emissions_kwh': emissions_kwh
    }

    print(f"\n--- Results for {model_name} ---")
    print(f" Duration: {total_duration_s:.2f}s | Emissions: {total_carbon_g / 1000:.6f} kg CO2eq | Queries: {num_queries}")
    print(json.dumps(results, indent=2))
    print("-" * 40)
    return results

# --- Main Experiment ---
if __name__ == "__main__":
    DEVICE = get_device()
    set_seed(42)
    batch_size = 16
    seq_len = 128
    num_layers = 1

    # Initialize results list
    all_results = []

    # Define model configurations
    model_configs = [
       # {'model_type': 'distilbert', 'task_type': 'classification', 'is_hybrid': False, 'name': 'Baseline_DistilBERT_Classification'},
      {'model_type': 'distilbert', 'task_type': 'classification', 'is_hybrid': True, 'name': 'Hybrid_DistilBERT_Classification'},
       #{'model_type': 'bert', 'task_type': 'classification', 'is_hybrid': False, 'name': 'Baseline_BERT_Classification'},
       # {'model_type': 'bert', 'task_type': 'classification', 'is_hybrid': True, 'name': 'Hybrid_BERT_Classification'},
    ]

    # Run experiments for each task separately to avoid interference
    for config in model_configs:
        model_type = config['model_type']
        task_type = config['task_type']
        is_hybrid = config['is_hybrid']
        #base_name = config['name']
        model_name = config['name']

        # Get dataloaders
        dataloaders, _, _, _, _ = get_dataloaders(model_type=model_type, seq_len=seq_len, batch_size=batch_size)

        # Run separate experiments for each task
        # for task in GLUE_TASKS_TO_RUN:
        #     model_name = f"{base_name}_{task.upper()}"
        #     print(f"\n=== Running experiment for {model_name} ===")

            # Build fresh model for each task
        if is_hybrid:

          model = build_hybrid_neuromorphic_model(
          device=DEVICE,
          model_type=model_type,
          task_type=task_type,
          num_layers=num_layers )
        else:
          model = build_baseline_model(
          device=DEVICE,
          model_type=model_type,
          task_type=task_type)


            # Run experiment
        results = run_experiment(
                model=model,
                model_name=model_name,
                dataloaders=dataloaders,
                device=DEVICE,
                is_bert=(model_type == 'bert'),
                batch_size=batch_size,
                seq_len=seq_len,
                task_type=task_type,
                  run_sst2=True, run_mrpc=True, run_rte=True
            )
        all_results.append(results)

    # Save all results to a JSON file
    with open('experiment_results.json', 'w') as f:
        json.dump(all_results, f, indent=2)

    df_out = pd.json_normalize(all_results, sep='_')
    df_out.to_csv('Hybrid_DistilBERTs.csv', index=False)
    print("✅ CSV saved to 'hybrid_gpu_neuromorphic_results.csv'")
    print("\nAll experiments completed. Results saved.")

Using device: cuda
✓ Using GPU: Tesla T4 (CUDA)
✓ Mixed precision (FP16) enabled.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

...

Epoch 3, Average Training Loss: 0.5639


Validating epoch 3: 100%|██████████| 18/18 [00:01<00:00, 11.98it/s]


Epoch 3, Average Validation Loss: 0.7108
No improvement in validation loss. Patience counter: 2/2
Early stopping triggered after 3 epochs.
Best Validation Loss for RTE: 0.6877
 Evaluating GLUE RTE...


Evaluating: 100%|██████████| 18/18 [00:01<00:00, 11.83it/s]

 RTE Accuracy: 0.5596, F1: 0.5588

--- Results for Hybrid_DistilBERT_Classification ---
 Duration: 656.69s | Emissions: 0.002041 kg CO2eq | Queries: 1557
{
  "model_name": "Hybrid_DistilBERT_Classification",
  "batch_size": 16,
  "seq_length": 128,
  "task_type": "classification",
  "accuracy_metrics": {
    "sst2_accuracy": 0.875,
    "sst2_f1": 0.8748086772528624,
    "mrpc_accuracy": 0.8333333333333334,
    "mrpc_f1": 0.8266201217329038,
    "rte_accuracy": 0.5595667870036101,
    "rte_f1": 0.5588393516948126
  },
  "performance_metrics": {
    "latency_ms_query": 421.76941088858365,
    "throughput_tokens_sec": 303.48336483276404,
    "energy_wh_token": 4.0963613490039166e-05,
    "sci_gco2e_query": 0.0013108356316812533,
    "wue_avg_liters_query": 9.438016548105022e-06,
    "total_emissions_kwh": 0.008163884314110845
  }
}
----------------------------------------
✅ CSV saved to 'hybrid_gpu_neuromorphic_results.csv'

All experiments completed. Results saved.
